# FFIG - A foreign function interface generator for `C++`

I want to write `C++` code and call it from Python without doing any extra work.

---

### For Project Managers
A language like Python can be written to resemble executable pseudo-code and is a great langauge for defining high-level acceptance criteria. 

Developer engagement is easy when requirements are code.

---

### For Clients 
It's easy to put together interactive demos with Python. Jupyter notebook allows one to display graphs, images, videos and tabulated data. 

The ability to change demos on the fly in response to client queries is seriously impressive.

---

### For Developers
I can break the edit-compile-link-test cycle with an interpreted scripting language. 

Once the design is right I can drive it into `C++`.

---

### Calling `C++` from Python

* Write some `C++` code.
* Define a `C-API` for binary compatibility.
* Solving problems with ownership.
* Define classes and functions that use a Foreign Function Interface to communicate with `C`.

---

### Code-generation

* Parse `C++` code.
* Transform parsed source code into a easy-to use form.
* Define a template that transforms parsed source code into `C`
* Define a template that transforms parsed source code into classes, functions and FFI calls.

---

### Extras

* Supporting Python 3.
* Supporting C++.

---

In [1]:
%%file Tree.hpp
#ifndef DEMOS_TREE_H
#define DEMOS_TREE_H

#include <memory>

class Tree {
  std::unique_ptr<Tree> left_;
  std::unique_ptr<Tree> right_;
  int data_ = 0;    
 
 public:
  Tree(int children) {
    if(children <=0) return; 
    left_ = std::make_unique<Tree>(children-1);
    right_ = std::make_unique<Tree>(children-1);
  } 
    
  Tree* left() { return left_.get(); }
  
  Tree* right() { return right_.get(); }
    
  int data() const { return data_; }
  
  void set_data(int x) { data_ = x; }
};

#endif // DEMOS_TREE_H

Overwriting Tree.hpp


Let's compile this to ensure we've not made any mistakes.

In [2]:
%%sh
clang++ -std=c++14 -fsyntax-only Tree.hpp

---

## Defining a C-API

We want a `C-API` so that we have a well-defined and portable binary interface. 

We'll have to re-model our code as C does not support classes.

Free functions with an extra leading argument for `this` should suffice.

In [3]:
%%file Tree_c.h

#define C_API extern "C" __attribute__((visibility("default")))

struct CTree_t;
typedef CTree_t* CTree;

C_API CTree Tree_create(int children);

C_API void Tree_dispose(CTree t);

C_API CTree Tree_left(CTree t);

C_API CTree Tree_right(CTree t);

C_API int Tree_data(CTree t);

C_API void Tree_set_data(CTree t, int x);

Overwriting Tree_c.h


In [4]:
%%file Tree_c.cpp

#include "Tree_c.h"
#include "Tree.hpp"

CTree Tree_create(int children) {
  auto tree = std::make_unique<Tree>(children);
  return reinterpret_cast<CTree>(tree.release());
}

void Tree_dispose(CTree t) {
  auto* tree = reinterpret_cast<Tree*>(t);
  delete tree;
}

CTree Tree_left(CTree t) {
  auto* tree = reinterpret_cast<Tree*>(t);
  return reinterpret_cast<CTree>(tree->left());
}

CTree Tree_right(CTree t) {
  auto* tree = reinterpret_cast<Tree*>(t);
  return reinterpret_cast<CTree>(tree->right());
}

int Tree_data(CTree t) {
  auto* tree = reinterpret_cast<Tree*>(t);
  return tree->data();
}

void Tree_set_data(CTree t, int x) {
  auto* tree = reinterpret_cast<Tree*>(t);
  tree->set_data(x);
}

Overwriting Tree_c.cpp


We can build this to create a shared library with our `C-API` symbols exposed.

In [5]:
%%file CMakeLists.txt

cmake_minimum_required(VERSION 3.0)
set(CMAKE_CXX_STANDARD 14)
add_compile_options(-fvisibility=hidden)
add_library(Tree_c SHARED Tree_c.cpp)

Overwriting CMakeLists.txt


In [6]:
%%sh
rm -rf CMakeFiles/
rm CMakeCache.txt

cmake . -GNinja
cmake --build .
strip -x libTree_c.dylib

-- The C compiler identification is AppleClang 8.1.0.8020042
-- The CXX compiler identification is AppleClang 8.1.0.8020042
-- Check for working C compiler: /Applications/Xcode.app/Contents/Developer/Toolchains/XcodeDefault.xctoolchain/usr/bin/cc
-- Check for working C compiler: /Applications/Xcode.app/Contents/Developer/Toolchains/XcodeDefault.xctoolchain/usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /Applications/Xcode.app/Contents/Developer/Toolchains/XcodeDefault.xctoolchain/usr/bin/c++
-- Check for working CXX compiler: /Applications/Xcode.app/Contents/Developer/Toolchains/XcodeDefault.xctoolchain/usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Configuring done
-- Generating done
-- Build files have been 

In [7]:
%%sh
nm -U libTree_c.dylib

0000000000001c30 T _Tree_create
0000000000001fc0 T _Tree_data
0000000000001ea0 T _Tree_dispose
0000000000001f00 T _Tree_left
0000000000001f60 T _Tree_right
0000000000002010 T _Tree_set_data


## Python interop

We can use Python's `ctypes` module to interact with the `C` shared-library.

In [8]:
import ctypes
ctypes.c_object_p = ctypes.POINTER(ctypes.c_void_p)

tree_lib = ctypes.cdll.LoadLibrary("libTree_c.dylib")

We need to tell ctypes about the arguments and return types of the functions.

By default ctypes assumes functions take no arguments and return an integer.

In [9]:
tree_lib.Tree_create.argtypes = [ctypes.c_int]
tree_lib.Tree_create.restype = ctypes.c_object_p

tree_lib.Tree_dispose.argtypes = [ctypes.c_object_p]
tree_lib.Tree_dispose.restype = None

tree_lib.Tree_left.argtypes = [ctypes.c_object_p]
tree_lib.Tree_left.restype = ctypes.c_object_p

tree_lib.Tree_right.argtypes = [ctypes.c_object_p]
tree_lib.Tree_right.restype = ctypes.c_object_p

tree_lib.Tree_data.argtypes = [ctypes.c_object_p]
tree_lib.Tree_data.restype = ctypes.c_int

tree_lib.Tree_set_data.argtypes = [ctypes.c_object_p, ctypes.c_int]
tree_lib.Tree_set_data.restype = None

We'll leave the string-related methods for now, interop there is not so easy.

Let's see what we can do with the fledgling Python API.

In [10]:
root = tree_lib.Tree_create(2)

In [11]:
tree_lib.Tree_data(root)

0

In [12]:
tree_lib.Tree_set_data(root, 42)
tree_lib.Tree_data(root)

42

In [13]:
tree_lib.Tree_dispose(root)

So far, so not-very-Pythonic.

We want classes!

In [14]:
class Tree(object):
    
    def __init__(self, children=None, _p=None):
        if _p:
            self._ptr = _p
            self._owner = False
        else:
            self._ptr = tree_lib.Tree_create(children)
            self._owner = True
        
    def __del__(self):
        if self._owner:
            tree_lib.Tree_dispose(self._ptr)
     
    def __repr__(self):
        return "<Tree data:{}>".format(self.data)
    
    @property
    def left(self):
        p = tree_lib.Tree_left(self._ptr)
        if not p: 
            return None
        return Tree(_p=p)
    
    @property
    def right(self):
        p = tree_lib.Tree_right(self._ptr)
        if not p: 
            return None
        
        return Tree(_p=p)
    
    @property
    def data(self):
        return tree_lib.Tree_data(self._ptr)
    
    @data.setter
    def data(self, x):
        tree_lib.Tree_set_data(self._ptr, x)

In [15]:
t = Tree(2)
t

<Tree data:0>

In [16]:
t.data = 42
t

<Tree data:42>

In [17]:
t.left

<Tree data:0>

In [18]:
t.left.data = 6

In [19]:
t.left

<Tree data:6>

This looks good but we our crude attempts at memory management will fail if we start working with temporaries.

In [20]:
# This kills the kernel
# left = Tree(3).left.left
# left.data

Our Python classes don't know enough about the underlying C++ to do the memory management. 

Our C-API implementation needs a re-think.

## Defining a C-API (Again)

We can imbue the pointers passed across the API boundary with object lifetime detail

### The aliasing constructor of `std::shared_ptr`

We can use the aliasing constructor of shared_ptr to keep objects alive while any subobject is exposed across the API boundary.

---

In [21]:
%%file Tree_2_c.cpp

#include <memory>
#include "Tree_c.h"
#include "Tree.hpp"

using Tree_ptr = std::shared_ptr<Tree>*;

CTree Tree_create(int children) {
  auto tree = std::make_unique<Tree>(children);
  Tree_ptr p = new std::shared_ptr<Tree>(tree.release());
  return reinterpret_cast<CTree>(p);
}

void Tree_dispose(CTree t) {
  delete reinterpret_cast<Tree_ptr>(t);
}

CTree Tree_left(CTree t) {
  const auto& tree = *reinterpret_cast<Tree_ptr>(t);
  auto left = tree->left();
  if(!left) 
    return nullptr;
    
  Tree_ptr p = new std::shared_ptr<Tree>(tree, left);
  return reinterpret_cast<CTree>(p);
}

CTree Tree_right(CTree t) {
  const auto& tree = *reinterpret_cast<Tree_ptr>(t);
  auto right = tree->left();
  if(!right) 
    return nullptr;
    
  Tree_ptr p = new std::shared_ptr<Tree>(tree, right);
  return reinterpret_cast<CTree>(p);
}

int Tree_data(CTree t) {
  const auto& tree = *reinterpret_cast<Tree_ptr>(t);
  return tree->data();
}

void Tree_set_data(CTree t, int x) {
  const auto& tree = *reinterpret_cast<Tree_ptr>(t);
  tree->set_data(x);
}

Overwriting Tree_2_c.cpp


In [22]:
%%file CMakeLists.txt

cmake_minimum_required(VERSION 3.0)
set(CMAKE_CXX_STANDARD 14)
add_compile_options(-fvisibility=hidden)
add_library(Tree_2_c SHARED Tree_2_c.cpp)

Overwriting CMakeLists.txt


In [23]:
%%sh
rm -rf CMakeFiles/
rm CMakeCache.txt

cmake . -GNinja
cmake --build .
strip -x libTree_2_c.dylib

-- The C compiler identification is AppleClang 8.1.0.8020042
-- The CXX compiler identification is AppleClang 8.1.0.8020042
-- Check for working C compiler: /Applications/Xcode.app/Contents/Developer/Toolchains/XcodeDefault.xctoolchain/usr/bin/cc
-- Check for working C compiler: /Applications/Xcode.app/Contents/Developer/Toolchains/XcodeDefault.xctoolchain/usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /Applications/Xcode.app/Contents/Developer/Toolchains/XcodeDefault.xctoolchain/usr/bin/c++
-- Check for working CXX compiler: /Applications/Xcode.app/Contents/Developer/Toolchains/XcodeDefault.xctoolchain/usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Configuring done
-- Generating done
-- Build files have been 

## A Safer Python API

In [24]:
import ctypes
ctypes.c_object_p = ctypes.POINTER(ctypes.c_void_p)

tree_lib2 = ctypes.cdll.LoadLibrary("libTree_2_c.dylib")

In [25]:
tree_lib2.Tree_create.argtypes = [ctypes.c_int]
tree_lib2.Tree_create.restype = ctypes.c_object_p

tree_lib2.Tree_dispose.argtypes = [ctypes.c_object_p]
tree_lib2.Tree_dispose.restype = None

tree_lib2.Tree_left.argtypes = [ctypes.c_object_p]
tree_lib2.Tree_left.restype = ctypes.c_object_p

tree_lib2.Tree_right.argtypes = [ctypes.c_object_p]
tree_lib2.Tree_right.restype = ctypes.c_object_p

tree_lib2.Tree_data.argtypes = [ctypes.c_object_p]
tree_lib2.Tree_data.restype = ctypes.c_int

tree_lib2.Tree_set_data.argtypes = [ctypes.c_object_p, ctypes.c_int]
tree_lib2.Tree_set_data.restype = None

In [26]:
class Tree2(object):
    
    def __init__(self, children=None, _p=None):
        if _p:
            self._ptr = _p
        else:
            self._ptr = tree_lib2.Tree_create(children)
        
    def __del__(self):
        tree_lib2.Tree_dispose(self._ptr)
     
    def __repr__(self):
        return "<Tree data:{}>".format(self.data)
    
    @property
    def left(self):
        p = tree_lib2.Tree_left(self._ptr)
        if not p: 
            return None
        return Tree2(_p=p)
    
    @property
    def right(self):
        p = tree_lib2.Tree_right(self._ptr)
        if not p: 
            return None
        
        return Tree2(_p=p)
    
    @property
    def data(self):
        return tree_lib2.Tree_data(self._ptr)
    
    @data.setter
    def data(self, x):
        tree_lib2.Tree_set_data(self._ptr, x)

In [27]:
# This no longer kills the kernel
left = Tree2(3).left.left.left
left.data

0

In [28]:
root = Tree2(3)
left = root.left
left.data = 42

del root
left.data

42

In addition to memory management, we have string translation and exception handling to think about.

Given time constraints, I won't cover that here. 

---

# FFIG

Writing C-API and Python bindings out by hand is time consuming and more than a little error prone.

There's not a lot of creativity required once the approach is worked out.

We want to generate it.

### Parsing `C++` with libclang

libclang has Python bindings and exposes enough of the AST that we can extract all the information we need.

In [29]:
import sys
sys.path.insert(0,'..')

import ffig.clang.cindex

index = ffig.clang.cindex.Index.create()
translation_unit = index.parse("Tree.h", ['-x', 'c++', '-std=c++14', '-I../ffig/include'])

In [30]:
import asciitree

def node_children(node):
    return (c for c in node.get_children() if c.location.file.name == "Tree.h")

print(asciitree.draw_tree(translation_unit.cursor,
  lambda n: [c for c in node_children(n)],
  lambda n: "%s (%s)" % (n.spelling or n.displayname, str(n.kind).split(".")[1])))

Tree.h (TRANSLATION_UNIT)
  +--Tree (CLASS_DECL)
     +--left_ (FIELD_DECL)
     |  +--std (NAMESPACE_REF)
     |  +--unique_ptr (TEMPLATE_REF)
     |  +--class Tree (TYPE_REF)
     +--right_ (FIELD_DECL)
     |  +--std (NAMESPACE_REF)
     |  +--unique_ptr (TEMPLATE_REF)
     |  +--class Tree (TYPE_REF)
     +--data_ (FIELD_DECL)
     |  +--std (NAMESPACE_REF)
     |  +--string (TYPE_REF)
     +-- (CXX_ACCESS_SPEC_DECL)
     +--Tree (CONSTRUCTOR)
     |  +--children (PARM_DECL)
     |  +-- (COMPOUND_STMT)
     |     +-- (IF_STMT)
     |     |  +-- (BINARY_OPERATOR)
     |     |  |  +--children (UNEXPOSED_EXPR)
     |     |  |  |  +--children (DECL_REF_EXPR)
     |     |  |  +-- (INTEGER_LITERAL)
     |     |  +-- (RETURN_STMT)
     |     +-- (UNEXPOSED_EXPR)
     |     |  +--operator= (CALL_EXPR)
     |     |     +--left_ (MEMBER_REF_EXPR)
     |     |     +--operator= (UNEXPOSED_EXPR)
     |     |     |  +--operator= (DECL_REF_EXPR)
     |     |     +-- (UNEXPOSED_EXPR)
     |     | 

---

We create some simple classes of our own to make handling the relevant AST info easy.

In [32]:
import ffig.cppmodel
import ffig.clang.cindex

model = ffig.cppmodel.Model(translation_unit)

In [33]:
[c.name for c in model.classes][-5:]

['char_traits',
 '__basic_string_common',
 'basic_string',
 'basic_string',
 'Tree']

In [34]:
["{} {}::{}".format(m.return_type, model.classes[-1].name, m.name) 
 for m in model.classes[-1].methods]

['Tree * Tree::left',
 'Tree * Tree::right',
 'const char * Tree::data',
 'void Tree::set_data']

---

### Jinja2 templates

Jinja2 is a lightweight web-templating engine used in Flask and we can use it to generate code from AST info.

In [36]:
from jinja2 import Template
template = Template(R"""
C++ 17 will bring us:
{%for feature in features%}
* {{feature}}
{% endfor%}
""")
print(template.render(
    {'features':['variant',
                 'optional',
                 'inline variables',
                 'fold-expressions',
                 'mandated-copy-elision']}))


C++ 17 will bring us:

* variant

* optional

* inline variables

* fold-expressions

* mandated-copy-elision



---

### Jinja2 and libclang

We can feed the AST info to a Jinja2 template to write some code for us.

In [37]:
tree_ast = model.classes[-1]

In [38]:
from jinja2 import Template
template = Template(R"""\
#ifndef   {{class.name|upper}}_H
#define   {{class.name|upper}}_H

struct {{class.name}}_t;
typedef {{class.name}}_t* {{class.name}};

{{class.name}} {{class.name}}_create();

{{class.name}} {{class.name}}_dispose({{class.name}} my{{class.name}});

{%- for m in class.methods %}{% if not m.arguments %}

{{class.name}} {{class.name}}_{{m.name}}({{class.name}} my{{class.name}});
{%- endif %}{% endfor %}

#else  // {{class.name|upper}}_H
#endif // {{class.name|upper}}_H

""")
print(template.render({'class':tree_ast}))

#ifndef   TREE_H
#define   TREE_H

struct Tree_t;
typedef Tree_t* Tree;

Tree Tree_create();

Tree Tree_dispose(Tree myTree);

Tree Tree_left(Tree myTree);

Tree Tree_right(Tree myTree);

Tree Tree_data(Tree myTree);

#else  // TREE_H
#endif // TREE_H



### Generating a Python API with FFIG

FFIG can be invoked to generate bindings for us.

FFIG requires that a class is annotated to create bindings for it.

In [39]:
%%file Shape.h
#include "ffig/attributes.h"
#include <stdexcept>
#include <string>

struct FFIG_EXPORT Shape {
  virtual ~Shape() = default;
  virtual double area() const = 0;
  virtual double perimeter() const = 0;
  virtual const char* name() const = 0;
};

static const double pi = 3.14159;

class Circle : public Shape {
  const double radius_;

public:
  double area() const override {
    return pi * radius_ * radius_;
  }

  double perimeter() const override {
    return 2 * pi * radius_;
  }

  const char* name() const override {
    return "Circle";
  }

  Circle(double radius) : radius_(radius) {
    if ( radius < 0 ) { 
      std::string s = "Circle radius \"" + std::to_string(radius_) + "\" must be non-negative.";
      throw std::runtime_error(s);
    }
  }
};

Overwriting Shape.h


In [56]:
%%sh
cd ../
python -m ffig -b rb.tmpl python -m Shape -i demos/Shape.h -o demos

In [41]:
%%sh
head Shape/_py3.py

# This code was generated by FFIG <http://ffig.org>.
# Manual edits will be lost.

import os
from ctypes import *
c_object_p = POINTER(c_void_p)

class c_interop_string(c_char_p):

  def __init__(self, p=None):


In [43]:
%cat Shape/_py3.py

# This code was generated by FFIG <http://ffig.org>.
# Manual edits will be lost.

import os
from ctypes import *
c_object_p = POINTER(c_void_p)

class c_interop_string(c_char_p):

  def __init__(self, p=None):
    if p is None:
      p = ""
    if isinstance(p, str):
      p = p.encode("utf8")
    super(c_char_p, self).__init__(p)

  def __str__(self):
    return self.value

  @property
  def value(self):
    if super(c_char_p, self).value is None:
      return None
    return super(c_char_p, self).value.decode("utf8")

  @classmethod
  def from_param(cls, param):
    if isinstance(param, str):
      return cls(param)
    if isinstance(param, bytes):
      return cls(param)
    raise TypeError("Cannot convert '{}' to '{}'".format(type(param).__name__, cls.__name__))

  @staticmethod
  def to_python_string(x, *args):
    return x.value


class Shape_error(Exception):
    def __init__(self):
        self.value = conf.lib.Shape_error()
        conf.lib.Shape_clear_error()

    def __str_

In [44]:
%%file CMakeLists.txt

cmake_minimum_required(VERSION 3.0)
set(CMAKE_CXX_STANDARD 14)

include_directories(../ffig/include)

add_library(Shape_c SHARED Shape_c.cpp)

Overwriting CMakeLists.txt


In [45]:
%%sh
rm -rf CMakeFiles/
rm CMakeCache.txt

cmake . -GNinja
cmake --build .
strip -x libAsset_c.dylib

-- The C compiler identification is AppleClang 8.1.0.8020042
-- The CXX compiler identification is AppleClang 8.1.0.8020042
-- Check for working C compiler: /Applications/Xcode.app/Contents/Developer/Toolchains/XcodeDefault.xctoolchain/usr/bin/cc
-- Check for working C compiler: /Applications/Xcode.app/Contents/Developer/Toolchains/XcodeDefault.xctoolchain/usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /Applications/Xcode.app/Contents/Developer/Toolchains/XcodeDefault.xctoolchain/usr/bin/c++
-- Check for working CXX compiler: /Applications/Xcode.app/Contents/Developer/Toolchains/XcodeDefault.xctoolchain/usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Configuring done
-- Generating done
-- Build files have been 

In [46]:
import Shape

c = Shape.Circle(8)

print("A {} with radius {} has area {}".format(c.name(), 8, c.area()))

A Circle with radius 8 has area 201.06176


In [51]:
%%script /opt/intel/intelpython27/bin/python

import Shape
Shape.Config.set_library_path(".")
c = Shape.Circle(8)

print("A {} with radius {} has area {}".format(c.name(), 8, c.area()))

A Circle with radius 8 has area 201.06176


In [52]:
Shape.Circle(-5)

Shape_error: Circle radius "-5.000000" must be non-negative.

In [57]:
%%ruby
load "Shape.rb"
c = Circle.new(8)

puts("A #{c.name()} with radius #{8} has area #{c.area()}")

A Circle with radius 8 has area 201.06176


## FFIG needs you!

FFIG is MIT-licensed and hosted on GitHub.

We'd _really_ like input on:

* Support for returning non-primitives by value
* Support for standard library types
* Generating boost::python bindings

Other issues are on: https://github.com/FFIG/ffig/issues

Our policy for PRs is to approve things that do not break existing code. No change is too small!